In [1]:
import os
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

from dataset.OnlineDataset import EnergyDataset
import torch

In [ ]:
ds = EnergyDataset('../data/1_processed',48,1,'eval')

prosumption = torch.tensor([])
price = torch.tensor([])
for i in range(0,1):
    prosumption = torch.cat([prosumption, ds[i]['prosumption']])
    price = torch.cat([price, ds[i]['price']])
T = len(prosumption)

# WO Battery

In [12]:
cost = 0
for t in range(T):
    cost += prosumption[t]*price[t] if prosumption[t]>=0 else prosumption[t]*0.1

print(cost)

tensor(3.4202)


# Gurobi Optimizer

In [ ]:

max_charge = 1.25
max_discharge = -1.25
battery_capacity = 10.0
big_M = 1e4  # A sufficiently large number

# Create model
model = gp.Model("BatteryCharging")
model.Params.OutputFlag = 1

# Variables
action = model.addVars(T, lb=max_discharge, ub=max_charge, name="action")
soe = model.addVars(T + 1, lb=0.0, ub=battery_capacity, name="soe")
z = model.addVars(T, lb=0.0, ub=10, vtype=GRB.CONTINUOUS, name='actual_price')
b = model.addVars(T, vtype=GRB.BINARY, name="b")

# Initial SoE
model.addConstr(soe[0] == 0.0, name="initial_soe")

eps = 0.0001
M = 20 + eps

for t in range(T):
    # Battery dynamics
    model.addConstr(soe[t + 1] == soe[t] + action[t], name=f"soe_update_{t}")
    model.addConstr(action[t]+prosumption[t] >= 0 + eps - M * (1 - b[t]), name="bigM_constr1")
    model.addConstr(action[t]+prosumption[t] <= 0 + M * b[t], name="bigM_constr2")

    # Add indicator constraints
    model.addConstr((b[t] == 1) >> (z[t] == price[t]), name="indicator_constr1")
    model.addConstr((b[t] == 0) >> (z[t] == 0.1), name="indicator_constr2")



# Objective function: minimize cost
obj = gp.quicksum(
    (prosumption[t] + action[t]) * z[t]
    for t in range(T)
)
model.setObjective(obj, GRB.MINIMIZE)

# Optimize
model.optimize()

if model.status == GRB.OPTIMAL:
    final_cost = 0.0
    actions = [action[t].X for t in range(T)]
    soes = [soe[t].X for t in range(T+1)]
    used_price = [z[t].X for t in range(T)]
    for t in range(T):
        curr_price = price[t] if (prosumption[t]+actions[t]) >= 0 else 0.1
        final_cost += (prosumption[t]+actions[t])*curr_price

    print(f'final cost: {final_cost}')

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 3700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 145 rows, 193 columns and 337 nonzeros
Model fingerprint: 0x870f5cb3
Model has 48 quadratic objective terms
Model has 96 simple general constraints
  96 INDICATOR
Variable types: 145 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-03, 2e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [4e-03, 2e+01]
  GenCon rhs range [1e-01, 4e-01]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 7 rows and 54 columns
Presolve time: 0.01s
Presolved: 184 rows, 185 columns, 460 nonzeros
Variable types: 139 continuous, 46 integer (46 binary)
Found heuristic solution: objective 2.3390627

Root relaxat